In [1]:
# Librerías:
import pandas as pd
import pyreadstat
import numpy as np

# Preparación de los datos:
# 1. Lectura dataset
df, meta = pyreadstat.read_sav("GEM 2021 NES GLOBAL INDIVIDUAL LEVEL NATIONS.sav")

# 2. Diccionario de etiquetas de valores
value_labels = meta.value_labels  # {'label_name': {code: label, ...}, ...}

# 3. Aplica mapeo solo a columnas con etiquetas y tipo entero (categóricas)
# Lista de columnas a categoricas a mapear
cols_a_mapear = ['NES_COUNTRY', 'NES_GENDER', 'NES_EDU', 'NES_PSPE', 'NES21_A09', 'NES21_A10']

for col in cols_a_mapear:
    if col in df.columns and col in meta.variable_to_label:
        label_name = meta.variable_to_label[col]
        if label_name in value_labels:
            df[col] = df[col].map(value_labels[label_name])

print(df.head())

# 2. Limpieza y formato
# Convertir fechas
df['END_DATE'] = pd.to_datetime(df['END_DATE'], errors='coerce')

# Dado que en los datasets GEM, los valores faltantes se codifican como ??????????, vamos a tratarlo:
# Reemplaza por NaN cualquier valor que contenga al menos un "?"
df['NES_PSPE'] = df['NES_PSPE'].apply(lambda x: np.nan if isinstance(x, str) and '?' in x else x)

# Eliminar columnas completamente vacías
df.dropna(axis=1, how='all', inplace=True)

# Eliminar filas sin país ni género o con errores de entrada evidentes
df = df[df['NES_COUNTRY'].notna()]
df = df[df['NES_GENDER'].notna()]
df = df[df['NES_AGE'].notna()]

# Paremitrización Age
df['NES_AGE'] = df['NES_AGE'] / 10

# 3. Selección de variables clave (seleccionadas en la práctica I)
vars_clave = [
    'NES_COUNTRY', 'NES_GENDER', 'NES_AGE', 'NES_EDU', 'NES_PSPE',
    'NES21_A04', 'NES21_A05', 'NES21_A06', 'NES21_A08', 'NES21_A03',
    'NES21_A09', 'NES21_A10', 'NES21_A02', 'NES21_A07'
]

df_sub = df[vars_clave].copy()

# Renombramos para mayor claridad
df_sub.rename(columns={
    'NES_COUNTRY' : 'Country',
    'NES_GENDER': 'Gender',
    'NES_AGE': 'Age',
    'NES_EDU': 'EducationLevel',
    'NES_PSPE': 'IncomeLevel',
    'NES21_A04': 'SelfPerceivedCapability',
    'NES21_A05': 'FearOfFailure',
    'NES21_A06': 'EntrepreneurEducation',
    'NES21_A08': 'AccessToFinance',
    'NES21_A03': 'PerceivedOpportunities',
    'NES21_A09': 'OpportunityMotivation',
    'NES21_A10': 'NecessityMotivation',
    'NES21_A02': 'KnowsEntrepreneurs',
    'NES21_A07': 'MediaAttention'
}, inplace=True)

# Convertir a numérico por si hay strings o NaNs
binary_vars = [
    'SelfPerceivedCapability', 'FearOfFailure', 'EntrepreneurEducation',
    'AccessToFinance', 'PerceivedOpportunities', 'KnowsEntrepreneurs', 'MediaAttention'
]
df_sub[binary_vars] = df_sub[binary_vars].apply(pd.to_numeric, errors='coerce')

# 4. Creación del índice de ecosistema emprendedor (IEE)
df_sub['IEE'] = df_sub[binary_vars].sum(axis=1) / len(binary_vars) * 100

def clasificar_emprendedor(row):
    if pd.notna(row['OpportunityMotivation']):
        return 'Por oportunidad'
    elif pd.notna(row['NecessityMotivation']):
        return 'Por necesidad'
    else:
        return 'No emprendedor'

df_sub['TipoEmprendedor'] = df_sub.apply(clasificar_emprendedor, axis=1)

# 5. Exportar para visualización
df_sub.to_csv("datos_preparados_GEM.csv", index=False)

ModuleNotFoundError: No module named 'pyreadstat'